# Visualização dos Resultados do Benchmark de LLMs com Plotly

Este notebook tem como objetivo analisar e visualizar os resultados gerados pelo script `run_benchmark.py`. Ele carrega o arquivo `benchmark_results.csv` e cria uma série de **gráficos interativos** para comparar o desempenho dos diferentes modelos de linguagem em termos de **pontuação (acurácia)** e **velocidade (duração)**.

### Seção 1: Configuração e Carregamento dos Dados

Primeiro, vamos importar as bibliotecas necessárias e carregar os dados do arquivo CSV para um DataFrame do Pandas.

In [33]:
import pandas as pd
import numpy as np
import plotly.express as px
import os

caminho_arquivo = "benchmark_results.csv"
if not os.path.exists(caminho_arquivo):
    print(f"Erro: O arquivo '{caminho_arquivo}' não foi encontrado.")
    print("Por favor, execute o benchmark primeiro ou verifique o caminho do arquivo.")
else:
    df = pd.read_csv(caminho_arquivo)
    display(df.head())

,model_name,test_case_id,difficulty,score,duration_seconds
0,dummy,easy-insert-01,easy,0,1.999986e-06
1,dummy,easy-insert-02,easy,0,1.499997e-06
2,dummy,easy-insert-03,easy,0,1.299995e-06
3,dummy,easy-insert-04,easy,0,1.000008e-06
4,dummy,easy-insert-05,easy,0,9.999931e-07


#### Engenharia de Atributos

A coluna `test_case_id` contém o tipo de tarefa (`get` ou `insert`). Vamos extrair essa informação para uma nova coluna chamada `tipo_tarefa` para facilitar análises futuras.

In [34]:
if "df" in locals():
    df["tipo_tarefa"] = df["test_case_id"].apply(
        lambda x: "get" if "get" in x else "insert"
    )
    print("Coluna 'tipo_tarefa' criada com sucesso.")
    display(df.head())

Coluna 'tipo_tarefa' criada com sucesso.


,model_name,test_case_id,difficulty,score,duration_seconds,tipo_tarefa
0,dummy,easy-insert-01,easy,0,1.999986e-06,insert
1,dummy,easy-insert-02,easy,0,1.499997e-06,insert
2,dummy,easy-insert-03,easy,0,1.299995e-06,insert
3,dummy,easy-insert-04,easy,0,1.000008e-06,insert
4,dummy,easy-insert-05,easy,0,9.999931e-07,insert


### Seção 2: Visão Geral do Desempenho

Nesta seção, criamos gráficos que resumem o desempenho geral de cada modelo, fornecendo uma visão macro da pontuação e velocidade.

In [35]:
# Gráfico 1: Pontuação Média Geral por Modelo
if "df" in locals():
    avg_score = df.groupby("model_name")["score"].mean().reset_index()
    fig = px.bar(
        avg_score,
        x="model_name",
        y="score",
        color="model_name",
        title="Pontuação Média Geral por Modelo",
        labels={"model_name": "Modelo", "score": "Pontuação Média"},
        color_discrete_sequence=px.colors.qualitative.Plotly,
    )

    fig.update_layout(
        title_font_size=20,
        title_x=0.5,
        xaxis_title_font_size=14,
        yaxis_title_font_size=14,
        showlegend=False,
    )
    fig.update_xaxes(tickangle=45)
    fig.show()

In [36]:
# Gráfico 2: Duração Média (Velocidade) por Modelo
if "df" in locals():
    avg_duration = df.groupby("model_name")["duration_seconds"].mean().reset_index()
    fig = px.bar(
        avg_duration,
        x="model_name",
        y="duration_seconds",
        color="model_name",
        title="Duração Média (Velocidade) por Modelo",
        labels={"model_name": "Modelo", "duration_seconds": "Duração Média (segundos)"},
        color_discrete_sequence=px.colors.sequential.Plasma,
    )

    fig.update_layout(
        title_font_size=20,
        title_x=0.5,
        xaxis_title_font_size=14,
        yaxis_title_font_size=14,
        showlegend=False,
    )
    fig.update_xaxes(tickangle=45)
    fig.show()

### Seção 3: Análise de Eficiência (Pontuação vs. Velocidade)

Um bom modelo não é apenas preciso, mas também rápido. O gráfico de dispersão abaixo nos ajuda a visualizar o trade-off entre pontuação e velocidade. O modelo ideal estaria no quadrante superior esquerdo (alta pontuação, baixa duração).

In [37]:
if "df" in locals():
    summary_df = (
        df.groupby("model_name")
        .agg(avg_score=("score", "mean"), avg_duration=("duration_seconds", "mean"))
        .reset_index()
    )
    fig = px.scatter(
        summary_df,
        x="avg_duration",
        y="avg_score",
        text="model_name",
        color="model_name",
        size=[20] * len(summary_df),
        title="Gráfico de Eficiência: Pontuação Média vs. Duração Média",
        labels={
            "avg_duration": "Duração Média (segundos) - Mais rápido →",
            "avg_score": "Pontuação Média - Melhor →",
        },
    )
    fig.update_traces(textposition="top center")
    fig.update_layout(
        title_font_size=20,
        title_x=0.5,
        xaxis_title_font_size=14,
        yaxis_title_font_size=14,
        legend_title="Modelos",
    )
    fig.show()

### Seção 4: Análise Detalhada do Desempenho

Agora, vamos aprofundar a análise, segmentando o desempenho dos modelos por nível de dificuldade, consistência e tipo de tarefa.

In [38]:
if "df" in locals():
    fig = px.bar(
        df,
        x="model_name",
        y="score",
        color="difficulty",
        barmode="group",
        category_orders={"difficulty": ["easy", "medium", "hard"]},
        title="Desempenho por Nível de Dificuldade",
        labels={
            "model_name": "Modelo",
            "score": "Pontuação Média",
            "difficulty": "Dificuldade",
        },
        color_discrete_map={"easy": "#636EFA", "medium": "#EF553B", "hard": "#00CC96"},
    )

    fig.update_layout(
        title_font_size=20,
        title_x=0.5,
        xaxis_title_font_size=14,
        yaxis_title_font_size=14,
    )
    fig.update_xaxes(tickangle=45)
    fig.show()

O gráfico de caixas (boxplot) abaixo mostra a distribuição das pontuações para cada modelo. Ele é excelente para avaliar a **consistência**. Um modelo com uma caixa curta e poucos pontos fora (outliers) é mais consistente e previsível do que um com uma caixa longa.

In [39]:
if "df" in locals():
    fig = px.box(
        df,
        x="model_name",
        y="score",
        color="model_name",
        title="Consistência do Desempenho (Distribuição de Pontuações)",
        labels={"model_name": "Modelo", "score": "Pontuação"},
    )

    fig.update_layout(
        title_font_size=20,
        title_x=0.5,
        xaxis_title_font_size=14,
        yaxis_title_font_size=14,
        showlegend=False,
    )
    fig.update_xaxes(tickangle=45)
    fig.show()

In [40]:
if "df" in locals():
    fig = px.bar(
        df,
        x="model_name",
        y="score",
        color="tipo_tarefa",
        barmode="group",
        title="Desempenho por Tipo de Tarefa (Get vs. Insert)",
        labels={
            "model_name": "Modelo",
            "score": "Pontuação Média",
            "tipo_tarefa": "Tipo de Tarefa",
        },
        color_discrete_sequence=px.colors.qualitative.Set2,
    )

    fig.update_layout(
        title_font_size=20,
        title_x=0.5,
        xaxis_title_font_size=14,
        yaxis_title_font_size=14,
    )
    fig.update_xaxes(tickangle=45)
    fig.show()